# Import Libraries

In [1]:
import os
import pickle
import pandas as pd
import networkx as nx
from collections import Counter
import random

import data_params as input_data_params

# Get Explanation Paths

In [2]:
DISEASE_PREFIX = input_data_params.disease
assert DISEASE_PREFIX == 'dmd' or 'hd' or 'oi'

DATASET_PREFIXES = ['prev', 'restr']
embedding_method = 'e2v'

seeded_emb = False

if seeded_emb:
    fixed_emb = '_seeded'
else:
    fixed_emb = ''

expl_folders = input_data_params.expl_folders
explanations_per_dataset = {}
explanation_pairs_per_dataset = {}

curr_working_dir = os.getcwd()
curr_output_dir = os.path.join(curr_working_dir, 'output', DISEASE_PREFIX)

chosen_explanations_per_dataset = {}

for dataset_prefix in DATASET_PREFIXES:
    nodes = pd.read_csv(f'../output/{DISEASE_PREFIX}/{dataset_prefix}_{DISEASE_PREFIX}_indexed_nodes.csv')
    
    dataset_output_dir = os.path.join(curr_output_dir, f'{dataset_prefix}_{embedding_method}{fixed_emb}', expl_folders[dataset_prefix])
    print(dataset_output_dir)
    
    all_explanations = []
    all_graphs = []
    all_pairs = []
    for item in os.listdir(dataset_output_dir):
        if 'incomplete' not in item and '.gpickle' in item:
            
            with open(os.path.join(dataset_output_dir, item), 'rb') as f:
                G = pickle.load(f)
                all_graphs.append(G)
                
            file_name_explanation = item.split('_graph.gpickle')[0]
            all_explanations.append(file_name_explanation)
            pair_file_name = f'{file_name_explanation}_pair.pkl'
            
            with open(os.path.join(dataset_output_dir, pair_file_name), 'rb') as f:
                loaded_info = pickle.load(f)
                
                symptom_index = nodes.loc[nodes['id'] == loaded_info['symptom_id']].index[0]
                symptom_label = nodes.loc[nodes['id'] == loaded_info['symptom_id']]['label'].values[0]
                
                drug_index = nodes.loc[nodes['id'] == loaded_info['drug_id']].index[0]
                drug_label = nodes.loc[nodes['id'] == loaded_info['drug_id']]['label'].values[0]
                
                all_pairs.append([f'{symptom_label} {symptom_index}', f'{drug_label} {drug_index}'])
            
    explanations_per_dataset[dataset_prefix] = all_graphs
    explanation_pairs_per_dataset[dataset_prefix] = all_pairs

c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g1_e2v\expl_5
c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g2_e2v\expl_all


# Number of Types of Edges, Nodes, Metapaths

In [7]:
def get_occurrence_df(c, label_name):
    c_perc = [{label_name: i, 'Percentage': c[i] / c.total() * 100.0, 'Appearances': c[i]} for i in c]
    c_df = pd.DataFrame.from_dict(c_perc)
    c_df['Percentage'] = c_df['Percentage'].transform(lambda x: '{:,.2f}%'.format(x))
    c_df = c_df.sort_values(by = ['Appearances'], ascending = False)
    return c_df

def count_occurrences(G):    
    node_types = []
    for n, attr in G.nodes(data=True):
        node_types.append(attr['type'])
        
    edge_types = []
    triplets = []
    for n1, n2, attr in G.edges(data=True):
        edge_type = attr['label']
        edge_types.append(edge_type)
        
        n1_type = G.nodes[n1]['type']
        n2_type = G.nodes[n2]['type']
        
        triplet = tuple([n1_type, edge_type, n2_type])
        triplets.append(triplet)
    
    node_types_df = get_occurrence_df(Counter(node_types), 'Node Type')
    
    edge_types_df = get_occurrence_df(Counter(edge_types), 'Edge Type')
    
    metapaths_df = get_occurrence_df(Counter(triplets), 'Metapath')
    
    return node_types_df, edge_types_df, metapaths_df

def get_shortest_path_len_drug_symptom(n1, n2, G):
    path_len = nx.shortest_path_length(G, n1, n2)
    return path_len

In [11]:
for dataset_prefix in DATASET_PREFIXES:
    node_types_per_expl = []
    edge_types_per_expl = []
    triplet_types_per_expl = []
    path_len_per_expl = []
    
    unique_pairs = []
    
    nr_expl = 0
    for expl_g, pair in zip(explanations_per_dataset[dataset_prefix], explanation_pairs_per_dataset[dataset_prefix]):
        node_types_df, edge_types_df, metapaths_df = count_occurrences(expl_g)
        
        nr_node_types = node_types_df['Node Type'].nunique()
        node_types_per_expl.append(nr_node_types)
        
        nr_edge_types = edge_types_df['Edge Type'].nunique()
        edge_types_per_expl.append(nr_edge_types)
        
        nr_triplet_types = metapaths_df['Metapath'].nunique()
        triplet_types_per_expl.append(nr_triplet_types)
        
        node_1, node_2 = pair
        shortest_path_len = get_shortest_path_len_drug_symptom(node_1, node_2, expl_g)
        path_len_per_expl.append(shortest_path_len)
        
        unique_pairs.append(tuple(pair))
        
        nr_expl += 1
    
    print('All drug-symptom pairs explained in the explanations found from dataset', dataset_prefix, set(unique_pairs))
    
    print(f'For the {nr_expl} explanations generated from dataset {dataset_prefix}')
    print(f'Average number of node types: {(sum(node_types_per_expl)/len(node_types_per_expl))}')
    print(f'Average number of edge types: {(sum(edge_types_per_expl)/len(edge_types_per_expl))}')
    print(f'Average number of triplets: {(sum(triplet_types_per_expl)/len(triplet_types_per_expl))}')
    print(f'Average shortest path length between drug and symptom pair: {(sum(path_len_per_expl)/len(path_len_per_expl))}')

All drug-symptom pairs explained in the explanations found from dataset 1 {('Delayed speech and language development 6575', 'neratinib 1772'), ('Global developmental delay 7687', 'neratinib 1772')}
For the 10 explanations generated from dataset 1
Average number of node types: 3.0
Average number of edge types: 4.5
Average number of triplets: 5.0
Average shortest path length between drug and symptom pair: 2.6
All drug-symptom pairs explained in the explanations found from dataset 2 {('Arrhythmia 4336', 'levosimendan 5262'), ('Cardiomyopathy 3222', 'levosimendan 5262')}
For the 17 explanations generated from dataset 2
Average number of node types: 4.647058823529412
Average number of edge types: 5.529411764705882
Average number of triplets: 8.058823529411764
Average shortest path length between drug and symptom pair: 3.2941176470588234
